#### Dependencies

In [44]:
import numpy as np
import csv
import os
import sys  # Import the sys module
import time
from functools import partial
from multiprocessing import Pool, cpu_count
from scipy.optimize import differential_evolution

sys.path.append('..')
from dataset_reader import Traces_Dataset
from DE_obj_model import de_obj_model   
from exp_hh_model import HH_model_exp

#### Read test dataset

In [97]:
dataset = Traces_Dataset('../dataset_test.csv')

params = dataset.params.numpy()
current_traces = dataset.current_traces.numpy()
time_traces = dataset.time_traces.numpy()

sample = 0
prestep_V_2d = dataset.prestep_V[sample].numpy().reshape(-1,1)
step_Vs_2d = dataset.step_Vs[sample].numpy().reshape(-1,1)
t = time_traces[sample]

target_traces = current_traces[sample]
target_params = params[sample]

In [98]:
# sim setup for obj evaluation model
sim_setup_2d = {'prestep_V': prestep_V_2d, 'step_Vs': step_Vs_2d, 't': t}

# model = de_obj_model(target_params, sim_setup_2d)

In [99]:
# sim_traces = model.simulation()
# sim_traces.shape

#### Plotting function for trail, target, and real data

In [20]:
def plot_trail_target_real(sim_setup_1d, t, sim_traces, target_traces, target_params): 
    '''
    sim_setup_1d is the exp setup for hh model, all default as in hh class
    t is the target time traces from generated dataset
    sim_traces is the generated during searching stage
    target_traces is the target current traces from the generated dataset
    target_params is the target params from the generated dataset, dict
    '''
    # compare samples and simulations using sample params
    hh_model = HH_model_exp(target_params, sim_setup_1d)
    current_traces_sim = hh_model.simulation()

    colors = ['blue', 'red', 'green', 'purple', 'orange', 'yellow', 'cyan', 'magenta', 'brown', 'gray', 'black']

    for step in range(t.shape[0]): 
        plt.plot(t[step], sim_traces[step], linestyle='-', color=colors[step])
        plt.plot(t[step], target_traces[step], linestyle=':', color=colors[step])
        plt.plot(sim_setup_1d['t'], current_traces_sim[step], linestyle='--', color=colors[step])
    plt.legend()
    plt.title(f"prestep_V: {sim_setup_1d['prestep_V']}; step_V1: {sim_setup_1d['step_Vs'][0]}")


In [21]:
# sim setup for plotting real data
sim_setup_1d = {'prestep_V': np.squeeze(prestep_V_2d), 'step_Vs': np.squeeze(step_Vs_2d), 't': np.arange(0.0, 6.0, 0.01)}
target_params_dict = {'p': target_params[0], 'g_max': target_params[1], 'E_rev': target_params[2], 'a_m': target_params[3], 'b_m': target_params[4], 'delta_m': target_params[5], 's_m': target_params[6]}
plot_trail_target_real(sim_setup_1d, t, sim_traces, target_traces, target_params_dict)

NameError: name 'sim_traces' is not defined

#### Define obj function

In [100]:
def obj(x, *args): 
    '''
    x: a 1-D array of the variables for the obj function (the parameters we are estimating)
    args: a tupleo f additional fixed parameters (prestep_V, step_V0, time_traces)
    *args=(sim_setup_2d, target_current_trances)
    '''
    trail_model = de_obj_model(x, args[0])
    trail_traces = trail_model.simulation()
    # print(trail_traces[1])
    target_model = de_obj_model(args[1], args[0])
    target_traces = target_model.simulation()
    # print(target_traces[1]) 

    fit = np.sum(np.square(trail_traces - target_traces))
    # relative_error = fit/np.sum(np.square(target_traces))
    
    return fit


In [101]:
fitness = obj(params[9], *(sim_setup_2d, target_params))
print(fitness)

2867943200.0


#### Define bounds

In [102]:
# these bounds are from the distribution of the params in the dataset used for NN training
params_searching_bounds = {
    'p': (1, 4),
    'g_max': (100, 140), 
    'E_rev': (-100, -60), 
    'a_m': (0, 100), 
    'b_m': (0, 100), 
    'delta_m': (0, 1), 
    's_m': (-100, 0)
}
bounds = [params_searching_bounds['p'], params_searching_bounds['g_max'], params_searching_bounds['E_rev'], params_searching_bounds['a_m'], params_searching_bounds['b_m'], params_searching_bounds['delta_m'], params_searching_bounds['s_m']]

In [103]:
# def callback_function(xk, convergence):
#     callback_function.iteration += 1
#     print("Iteration:", callback_function.iteration)
#     print("Solution:", xk)
#     print("Objective value:", obj(xk, sim_setup_2d, target_params))
#     print("------------------------")
# callback_function.iteration = 0  # Initialize the iteration counter

In [104]:
from scipy.optimize import differential_evolution

result = differential_evolution(obj, bounds, args=(sim_setup_2d, target_params),  seed=42, maxiter=1000, disp=True)#, callback=callback_function)

/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:63: RuntimeWarning: divide by zero encountered in divide
  return self.m_infty(V) + (self.m_infty(self.prestep_V) - self.m_infty(V)) * np.exp(- self.t / self.tau_m(V))
/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:63: RuntimeWarning: invalid value encountered in divide
  return self.m_infty(V) + (self.m_infty(self.prestep_V) - self.m_infty(V)) * np.exp(- self.t / self.tau_m(V))
/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:63: RuntimeWarning: overflow encountered in divide
  return self.m_infty(V) + (self.m_infty(self.prestep_

differential_evolution step 1: f(x)= 7.62291e+08
differential_evolution step 2: f(x)= 4.10959e+08
differential_evolution step 3: f(x)= 4.10959e+08
differential_evolution step 4: f(x)= 4.10959e+08
differential_evolution step 5: f(x)= 4.10959e+08
differential_evolution step 6: f(x)= 4.10959e+08
differential_evolution step 7: f(x)= 4.10959e+08
differential_evolution step 8: f(x)= 4.09484e+08
differential_evolution step 9: f(x)= 4.09484e+08
differential_evolution step 10: f(x)= 3.57067e+08
differential_evolution step 11: f(x)= 3.57067e+08
differential_evolution step 12: f(x)= 2.15659e+08
differential_evolution step 13: f(x)= 2.15659e+08
differential_evolution step 14: f(x)= 2.15659e+08
differential_evolution step 15: f(x)= 1.55005e+08
differential_evolution step 16: f(x)= 1.18615e+08
differential_evolution step 17: f(x)= 1.18615e+08
differential_evolution step 18: f(x)= 1.18615e+08
differential_evolution step 19: f(x)= 9.91241e+07
differential_evolution step 20: f(x)= 7.13698e+07
different

In [105]:
target_params, result.x

(array([  3.        , 129.03758   , -64.20662   ,   1.2536927 ,
         90.053795  ,   0.31965497,  -9.5933075 ], dtype=float32),
 array([  3.0000002 , 129.03833336, -64.20576178,   1.25369433,
         90.05387922,   0.31965474,  -9.5933048 ]))

In [91]:
(target_params - result.x) ** 2

array([8.51738132e-12, 4.00138186e-07, 6.63297270e-07, 1.89348800e-13,
       6.31993869e-10, 8.84772837e-14, 2.39252303e-10])

In [66]:
np.mean((target_params - result.x) ** 2)

1.151414216251836e-12

In [42]:
result.x

array([  3.81225427, 117.68948712, -80.57534375,   1.65482514,
        17.64026358,   0.248886  , -17.59442741])

In [ ]:
result

 message: Maximum number of iterations has been exceeded.
 success: False
     fun: 26.620414408339457
       x: [ 3.295e+00  1.067e+02 -7.884e+01  4.578e+00  1.433e+01
            1.639e-01 -4.363e+01]
     nit: 300
    nfev: 31797
     jac: [ 4.463e-02 -3.375e-04  3.823e-04  0.000e+00  0.000e+00
            0.000e+00  0.000e+00]

### test on all samples

In [115]:
dataset = Traces_Dataset('../dataset_test.csv')

params = dataset.params.numpy()
current_traces = dataset.current_traces.numpy()
time_traces = dataset.time_traces.numpy()

prestep_V_vec = dataset.prestep_V.numpy()
step_Vs_vec = dataset.step_Vs.numpy()

In [116]:
def obj(x, *args): 
    '''
    x: a 1-D array of the variables for the obj function (the parameters we are estimating)
    *args=(sim_setup_2d, target_current_trances)
    '''
    trail_model = de_obj_model(x, args[0])
    trail_traces = trail_model.simulation()
    # print(trail_traces[1])
    target_model = de_obj_model(args[1], args[0])
    target_traces = target_model.simulation()
    # print(target_traces[1]) 

    fit = np.sum(np.square(trail_traces - target_traces))
    # relative_error = fit/np.sum(np.square(target_traces))
    
    return fit

In [117]:
# these bounds are from the distribution of the params in the dataset used for NN training
params_searching_bounds = {
    'p': (1, 4),
    'g_max': (100, 140), 
    'E_rev': (-100, -60), 
    'a_m': (0, 100), 
    'b_m': (0, 100), 
    'delta_m': (0, 1), 
    's_m': (-100, 0)
}
bounds = [params_searching_bounds['p'], params_searching_bounds['g_max'], params_searching_bounds['E_rev'], params_searching_bounds['a_m'], params_searching_bounds['b_m'], params_searching_bounds['delta_m'], params_searching_bounds['s_m']]

In [118]:
mse_list = []
time_list = []

for sample in range(10):
    print(f'{sample}: \n')               
    prestep_V_2d = prestep_V_vec[sample].reshape(-1,1)
    step_Vs_2d = step_Vs_vec[sample].reshape(-1,1)
    t = time_traces[sample]

    # prestep_V_2d = dataset.prestep_V[sample].numpy().reshape(-1,1)
    # step_Vs_2d = dataset.step_Vs[sample].numpy().reshape(-1,1)
    # t = time_traces[sample]

    # target_traces = current_traces[sample]
    target_params = params[sample]

    # sim setup for obj evaluation model
    sim_setup_2d = {'prestep_V': prestep_V_2d, 'step_Vs': step_Vs_2d, 't': t}   

    start_time = time.time()
    result = differential_evolution(obj, bounds, args=(sim_setup_2d, target_params), seed=42, maxiter=1000, disp=True)
    end_time = time.time()

    print(target_params, result.x)

    mse = (target_params - result.x) ** 2
    mse_list.append(mse)
    elapsed_time = end_time - start_time
    time_list.append(elapsed_time)

mse_mat = np.vstack(mse_list)
time_mat = np.array(time_list).reshape(-1, 1)

mse_overall_avg = np.mean(mse_mat)
mse_overall_std = np.std(np.mean(mse_mat, axis=1))
time_overall_avg = np.mean(time_mat)
time_overall_std = np.std(time_mat)

0: 

differential_evolution step 1: f(x)= 7.62291e+08
differential_evolution step 2: f(x)= 4.10959e+08
differential_evolution step 3: f(x)= 4.10959e+08
differential_evolution step 4: f(x)= 4.10959e+08
differential_evolution step 5: f(x)= 4.10959e+08
differential_evolution step 6: f(x)= 4.10959e+08
differential_evolution step 7: f(x)= 4.10959e+08
differential_evolution step 8: f(x)= 4.09484e+08
differential_evolution step 9: f(x)= 4.09484e+08
differential_evolution step 10: f(x)= 3.57067e+08
differential_evolution step 11: f(x)= 3.57067e+08


/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:63: RuntimeWarning: divide by zero encountered in divide
  return self.m_infty(V) + (self.m_infty(self.prestep_V) - self.m_infty(V)) * np.exp(- self.t / self.tau_m(V))
/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:63: RuntimeWarning: invalid value encountered in divide
  return self.m_infty(V) + (self.m_infty(self.prestep_V) - self.m_infty(V)) * np.exp(- self.t / self.tau_m(V))
/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:63: RuntimeWarning: overflow encountered in divide
  return self.m_infty(V) + (self.m_infty(self.prestep_

differential_evolution step 12: f(x)= 2.15659e+08
differential_evolution step 13: f(x)= 2.15659e+08
differential_evolution step 14: f(x)= 2.15659e+08
differential_evolution step 15: f(x)= 1.55005e+08
differential_evolution step 16: f(x)= 1.18615e+08
differential_evolution step 17: f(x)= 1.18615e+08
differential_evolution step 18: f(x)= 1.18615e+08
differential_evolution step 19: f(x)= 9.91241e+07
differential_evolution step 20: f(x)= 7.13698e+07
differential_evolution step 21: f(x)= 7.13698e+07
differential_evolution step 22: f(x)= 4.90168e+07
differential_evolution step 23: f(x)= 4.74536e+07
differential_evolution step 24: f(x)= 3.84297e+07
differential_evolution step 25: f(x)= 3.84297e+07
differential_evolution step 26: f(x)= 3.84297e+07
differential_evolution step 27: f(x)= 3.84297e+07
differential_evolution step 28: f(x)= 3.84297e+07
differential_evolution step 29: f(x)= 3.84297e+07
differential_evolution step 30: f(x)= 3.84297e+07
differential_evolution step 31: f(x)= 3.84297e+07


/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:42: RuntimeWarning: overflow encountered in exp
  return ((tau_0m * np.exp(self.delta_m * ((V - self.V_2m) / self.s_m))) / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:42: RuntimeWarning: invalid value encountered in divide
  return ((tau_0m * np.exp(self.delta_m * ((V - self.V_2m) / self.s_m))) / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))


differential_evolution step 43: f(x)= 3.19932e+06
differential_evolution step 44: f(x)= 3.19932e+06
differential_evolution step 45: f(x)= 2.33906e+06
differential_evolution step 46: f(x)= 2.33906e+06
differential_evolution step 47: f(x)= 1.59174e+06
differential_evolution step 48: f(x)= 1.59174e+06
differential_evolution step 49: f(x)= 1.59174e+06
differential_evolution step 50: f(x)= 1.59174e+06
differential_evolution step 51: f(x)= 1.59174e+06
differential_evolution step 52: f(x)= 1.59174e+06
differential_evolution step 53: f(x)= 1.59174e+06
differential_evolution step 54: f(x)= 1.59174e+06
differential_evolution step 55: f(x)= 1.59174e+06
differential_evolution step 56: f(x)= 1.59174e+06
differential_evolution step 57: f(x)= 1.58302e+06
differential_evolution step 58: f(x)= 1.58302e+06
differential_evolution step 59: f(x)= 1.58302e+06
differential_evolution step 60: f(x)= 1.58302e+06
differential_evolution step 61: f(x)= 1.58302e+06
differential_evolution step 62: f(x)= 1.58302e+06


In [73]:
mse_overall_avg

127.9836217441734

In [89]:
target_params, result.x

(array([  4.       , 118.27517  , -71.81706  ,   2.0534372,  13.061292 ,
          0.1570644, -11.075991 ], dtype=float32),
 array([  3.99999708, 118.27580194, -71.81624795,   2.05343767,
         13.06131683,   0.1570641 , -11.07597521]))

### Hyperparameter Tuning

In [ ]:
dataset = Traces_Dataset('../dataset_test.csv')

params = dataset.params.numpy()
current_traces = dataset.current_traces.numpy()
time_traces = dataset.time_traces.numpy()

prestep_V_2d_vec = dataset.prestep_V.numpy().reshape(-1,1)
step_Vs_2d_vec = dataset.step_Vs.numpy().reshape(-1,1)

In [ ]:
def obj(x, *args): 
    '''
    x: a 1-D array of the variables for the obj function (the parameters we are estimating)
    *args=(sim_setup_2d, target_current_trances)
    '''
    trail_model = de_obj_model(x, args[0])
    trail_traces = trail_model.simulation()
    # print(trail_traces[1])
    target_model = de_obj_model(args[1], args[0])
    target_traces = target_model.simulation()
    # print(target_traces[1]) 

    fit = np.sum(np.square(trail_traces - target_traces))
    # relative_error = fit/np.sum(np.square(target_traces))
    
    return fit

In [ ]:
# these bounds are from the distribution of the params in the dataset used for NN training
params_searching_bounds = {
    'p': (1, 4),
    'g_max': (100, 140), 
    'E_rev': (-100, -60), 
    'a_m': (0, 100), 
    'b_m': (0, 100), 
    'delta_m': (0, 1), 
    's_m': (-100, 0)
}
bounds = [params_searching_bounds['p'], params_searching_bounds['g_max'], params_searching_bounds['E_rev'], params_searching_bounds['a_m'], params_searching_bounds['b_m'], params_searching_bounds['delta_m'], params_searching_bounds['s_m']]

In [ ]:
hyperparameters_grid = {
    'strategy': ['best1bin', 'best1exp', 'rand1exp', 'rand1exp', 
                'rand2bin', 'rand2exp', 'best2bin', 'best2exp',
                'randtobest1bin', 'randtobest1exp',
                'currenttobest1bin', 'currenttobest1exp'],
    'popsize': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],  # Example popsize hyperparameter
    'mutation': [0.5, (0.1, 0.9)],  # Example mutation hyperparameter
    'recombination': [0.1, 0.2, 0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9],  # Example recombination hyperparameter
    'init': ['random', 'sobol', 'latinhypercube'],  # Example init hyperparameter
}


In [ ]:
csv_filename = "de_experiment_results_try_2.csv"
# Define the headers for the CSV file
csv_headers = ['Strategy', 'Popsize', 'Mutation', 'Recombination', 'Init', 'MSE Overall Avg', 'MSE Overall Std', 'Elapsed Time Avg', 'Elapsed Time Std']

# Check if the CSV file exists; if not, create and write the headers
if not os.path.exists(csv_filename):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(csv_headers)

In [ ]:
cpu_count()

10

In [ ]:
def process_sample(sample, strategy, popsize, mutation, recombination, init):
    prestep_V_2d = prestep_V_2d_vec[sample]
    step_Vs_2d = step_Vs_2d_vec[sample]
    t = time_traces[sample]
    # target_traces = current_traces[sample]
    target_params = params[sample]

    # sim setup for obj evaluation model
    sim_setup_2d = {'prestep_V': prestep_V_2d, 'step_Vs': step_Vs_2d, 't': t}   

    start_time = time.time()
    result = differential_evolution(obj, bounds, args=(sim_setup_2d, target_params), strategy=strategy, popsize=popsize, mutation=mutation, recombination=recombination, init=init, seed=42, maxiter=300)
    end_time = time.time()
    
    mse = (target_params - result.x) ** 2
    elapsed_time = end_time - start_time
    return sample, mse, elapsed_time

if __name__ == '__main__':
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        # pool = Pool()  # Creates a pool of processes
        num_processes = cpu_count()
        
        # Create the pool with the specified number of processes
        pool = Pool(processes=num_processes)  
        
        for strategy in hyperparameters_grid['strategy']:
            for popsize in hyperparameters_grid['popsize']:
                for mutation in hyperparameters_grid['mutation']:
                    for recombination in hyperparameters_grid['recombination']:
                        for init in hyperparameters_grid['init']:
                            # Use partial to fix hyperparameters for the current loop iteration
                            process_func = partial(process_sample, strategy=strategy, popsize=popsize, mutation=mutation, recombination=recombination, init=init)
                            
                            # Map the process function to the sample range using the multiprocessing pool
                            results = pool.map(process_func, range(1000))

                            mse_list = []
                            time_list = []
                            for result in results:
                                sample, mse, elapsed_time = result
                                mse_list.append(mse)
                                time_list.append(elapsed_time)

                            mse_mat = np.vstack(mse_list)
                            time_mat = np.array(time_list).reshape(-1, 1)

                            mse_overall_avg = np.mean(mse_mat)
                            mse_overall_std = np.std(np.mean(mse_mat, axis=1))
                            time_overall_avg = np.mean(time_mat)
                            time_overall_std = np.std(time_mat)
                            
                            writer.writerow([strategy, popsize, mutation, recombination, init, mse_overall_avg, mse_overall_std, time_overall_avg, time_overall_std])
        
        pool.close()  # Close the pool
        pool.join()   # Wait for all processes to finish before exiting



Process SpawnPoolWorker-54:
Process SpawnPoolWorker-52:
Process SpawnPoolWorker-51:
Process SpawnPoolWorker-56:
Process SpawnPoolWorker-53:
Process SpawnPoolWorker-55:
Process SpawnPoolWorker-57:
Traceback (most recent call last):
  File "/Users/maxwellyue/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/maxwellyue/anaconda3/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/maxwellyue/anaconda3/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/maxwellyue/anaconda3/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'process_sample' on <module '__main__' (built-in)>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/maxwellyue/anaconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstra

KeyboardInterrupt: 

In [ ]:
with open(csv_filename, mode='a', newline='') as file:
    writer = csv.writer(file)
    for strategy in hyperparameters_grid['strategy']:
        for popsize in hyperparameters_grid['popsize']:
            for mutation in hyperparameters_grid['mutation']:
                for recombination in hyperparameters_grid['recombination']:
                    for init in hyperparameters_grid['init']:
                        mse_list = []
                        time_list = []
                        for sample in range(3): 
                            
                            prestep_V_2d = prestep_V_2d_vec[sample]
                            step_Vs_2d = step_Vs_2d_vec[sample]
                            t = time_traces[sample]
                            # target_traces = current_traces[sample]
                            target_params = params[sample]

                            # sim setup for obj evaluation model
                            sim_setup_2d = {'prestep_V': prestep_V_2d, 'step_Vs': step_Vs_2d, 't': t}   

                            start_time = time.time()
                            result = differential_evolution(obj, bounds, args=(sim_setup_2d, target_params), strategy=strategy, popsize=popsize, mutation=mutation, recombination=recombination, init=init, seed=42, maxiter=2)
                            end_time = time.time()
                        
                            mse = (target_params - result.x) ** 2
                            mse_list.append(mse)
                            elapsed_time = end_time - start_time
                            time_list.append(elapsed_time)

                        mse_mat = np.vstack(mse_list)
                        time_mat = np.array(time_list).reshape(-1, 1)

                        mse_overall_avg = np.mean(mse_mat)
                        mse_overall_std = np.std(np.mean(mse_mat, axis=1))
                        time_overall_avg = np.mean(time_mat)
                        time_overall_std = np.std(time_mat)
                        
                        writer.writerow([strategy, popsize, mutation, recombination, init, sample, mse_overall_avg, mse_overall_std, time_overall_avg, time_overall_std])

/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:63: RuntimeWarning: divide by zero encountered in divide
  return self.m_infty(V) + (self.m_infty(self.prestep_V) - self.m_infty(V)) * np.exp(- self.t / self.tau_m(V))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:63: RuntimeWarning: invalid value encountered in divide
  return self.m_infty(V) + (self.m_infty(self.prestep_V) - self.m_infty(V)) * np.exp(- self.t / self.tau_m(V))


KeyboardInterrupt: 

### Parallization

In [ ]:
import csv
import os
csv_filename = "de_experiment_results_parrallell_4.csv"
# Define the headers for the CSV file
csv_headers = ['Strategy', 'Popsize', 'Mutation', 'Recombination', 'Init', 'MSE Overall Avg', 'MSE Overall Std', 'Elapsed Time Avg', 'Elapsed Time Std']

# Check if the CSV file exists; if not, create and write the headers
if not os.path.exists(csv_filename):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(csv_headers)

In [ ]:
import csv
import time
import numpy as np
from functools import partial
from multiprocessing import Pool
from scipy.optimize import differential_evolution

def process_sample(sample, strategy, popsize, mutation, recombination, init):
    prestep_V_2d = prestep_V_2d_vec[sample]
    step_Vs_2d = step_Vs_2d_vec[sample]
    t = time_traces[sample]
    # target_traces = current_traces[sample]
    target_params = params[sample]

    # sim setup for obj evaluation model
    sim_setup_2d = {'prestep_V': prestep_V_2d, 'step_Vs': step_Vs_2d, 't': t}   

    start_time = time.time()
    result = differential_evolution(obj, bounds, args=(sim_setup_2d, target_params), strategy=strategy, popsize=popsize, mutation=mutation, recombination=recombination, init=init, seed=42, maxiter=2)
    end_time = time.time()
    
    mse = (target_params - result.x) ** 2
    elapsed_time = end_time - start_time
    return sample, mse, elapsed_time

if __name__ == '__main__':
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        pool = Pool()  # Creates a pool of processes
        
        for strategy in hyperparameters_grid['strategy']:
            for popsize in hyperparameters_grid['popsize']:
                for mutation in hyperparameters_grid['mutation']:
                    for recombination in hyperparameters_grid['recombination']:
                        for init in hyperparameters_grid['init']:
                            # Use partial to fix hyperparameters for the current loop iteration
                            process_func = partial(process_sample, strategy=strategy, popsize=popsize, mutation=mutation, recombination=recombination, init=init)
                            
                            # Map the process function to the sample range using the multiprocessing pool
                            results = pool.map(process_func, range(3))

                            mse_list = []
                            time_list = []
                            for result in results:
                                sample, mse, elapsed_time = result
                                mse_list.append(mse)
                                time_list.append(elapsed_time)

                            mse_mat = np.vstack(mse_list)
                            time_mat = np.array(time_list).reshape(-1, 1)

                            mse_overall_avg = np.mean(mse_mat)
                            mse_overall_std = np.std(np.mean(mse_mat, axis=1))
                            time_overall_avg = np.mean(time_mat)
                            time_overall_std = np.std(time_mat)
                            
                            writer.writerow([strategy, popsize, mutation, recombination, init, mse_overall_avg, mse_overall_std, time_overall_avg, time_overall_std])
        
        pool.close()  # Close the pool
        pool.join()   # Wait for all processes to finish before exiting


/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized

KeyboardInterrupt: 

In [ ]:
np.mean(mse_mat, axis=1).shape

(439,)

In [ ]:
np.mean(mse_mat), np.std(np.mean(mse_mat, axis=1)), np.mean(time_mat), np.std(time_mat)

(511.3364727433909, 417.878264540534, 0.1499422720733156, 0.033234656515269134)

In [ ]:
np.mean(np.mean(mse_mat, axis=0)), np.mean(time_mat, axis=0)

(511.336472743391, array([0.14994227]))

In [ ]:
time_mat.shape, mse_mat.shape

((439, 1), (439, 7))

In [ ]:
np.hstack((mse_mat, time_array.reshape(-1, 1)))

(220, 8)

#### Run DE on all test dataset, check result mse on all params here

In [ ]:
mse_mat = np.load('de_mse_params_all_test.npy')

In [ ]:
mse_mat.shape

(30, 7)

In [ ]:
mse_params = np.mean(mse_mat, axis=0)
mse_params

array([1.68353114e-03, 5.70682053e+01, 1.12901201e+02, 2.79381230e-03,
       7.58826951e+00, 1.83145674e-04, 7.30552287e-02])

## Define obj fct and estimate for single param

In [ ]:
def obj_i_param(x, *args): 
    '''
    x: a single float value of the i-th param
    args: a tupleo f additional fixed parameters (prestep_V, step_V0, time_traces)
    *args=(i, sim_setup_2d, target_params)
    '''
    params = args[2].copy()
    params[args[0]] = x
    trail_model = de_obj_model(params, args[1])
    trail_traces = trail_model.simulation()
    # print(trail_traces[1])
    target_model = de_obj_model(args[2], args[1])
    target_traces = target_model.simulation()
    # print(target_traces[1]) 
    
    return np.sum(np.square(trail_traces - target_traces))

In [ ]:
params_searching_bounds = {
    'p': (1, 5),
    'g_max': (100, 140), 
    'E_rev': (-100, -60), 
    'a_m': (0, 13), 
    'b_m': (0, 100), 
    'delta_m': (0, 1), 
    's_m': (-17, -10)
}
i = 6
key_at_index = list(params_searching_bounds.keys())[i]
bounds = [params_searching_bounds[key_at_index]]
bounds

[(-17, -10)]

In [ ]:
from scipy.optimize import differential_evolution
result = differential_evolution(obj_i_param, bounds, args=(i, sim_setup_2d, target_params), maxiter=100)#, tol = 1e-10)
result

 message: Optimization terminated successfully.
 success: True
     fun: 0.0
       x: [-1.105e+01]
     nit: 22
    nfev: 347

In [ ]:
((target_params[i] - result.x) ** 2)

array([2.0789837e-15])

#### Results for all single param estimation

In [ ]:
mses = np.load('de_mses_on_ith_params.npy')
nits = np.load('de_nit_on_ith_params.npy')
mses.shape, nits.shape

((30, 7), (30, 7))

In [ ]:
arr = mses.mean(axis=0)
print(arr)

sorted_indices = np.argsort(arr)

print("Sorted indices:", sorted_indices)

[1.40752475e-15 6.77700377e-15 2.20245039e-15 1.40308669e-15
 2.14625476e-15 1.44141350e-15 2.20503233e-15]
Sorted indices: [3 0 5 4 2 6 1]


In [ ]:
arr = nits.mean(axis=0)
print(arr)

sorted_indices = np.argsort(arr)

print("Sorted indices:", sorted_indices)

[27.         26.1        26.53333333 26.53333333 26.83333333 27.23333333
 26.33333333]
Sorted indices: [1 6 2 3 4 0 5]
